In [ ]:
from ml_project.utils import files,paths
from tqdm.notebook import tqdm

# new_tile for duplicate tiles to False

points_df = files.load_points_df()

points_df.loc[points_df.duplicated(subset=['tile_x','tile_y'],keep='first')&points_df['new_tile'],'new_tile'] = False

files.save_points_df(points_df,override=True)

# Correct the Order of the columns

files.save_points_df(files.load_points_df(),override=True)

files.save_samples_df(files.load_samples_df(),override=True)

# Add filenames

samples_df = files.load_samples_df()

for i,sample in samples_df.iterrows():
    samples_df.loc[i,'satellite_file'] = paths.FILE_SATELLITE_IMAGE(sample['tile_x'],sample['tile_y'],sample['tile_z']).name
    samples_df.loc[i,'mask_file'] = paths.FILE_MASK_IMAGE(sample['tile_x'],sample['tile_y'],sample['tile_z']).name

files.save_samples_df(samples_df,override=True)

# Update filenames

samples_df = files.load_samples_df()

for i,sample in tqdm(samples_df.iterrows()):
    x = sample['tile_x']
    y = sample['tile_y']
    z = sample['tile_z']
    
    old_satellite = paths.SATELLITE_IMAGES / sample['satellite_file']
    old_mask = paths.MASK_IMAGES / sample['mask_file']
    
    new_satellite = paths.FILE_SATELLITE_IMAGE(x,y,z)
    new_mask = paths.FILE_MASK_IMAGE(x,y,z)
    
    old_satellite.rename(new_satellite)
    old_mask.rename(new_mask)
    
    samples_df.loc[i,'satellite_file'] = new_satellite.name
    samples_df.loc[i,'mask_file'] = new_mask.name

files.save_samples_df(samples_df,override=True)

In [ ]:
files.load_samples_df()